In [ ]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt
import yaml
import glob

# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

# matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature

# Import my modules
sys.path.append('../modules') # Path to modules
import ar_funcs
from plotter import draw_basemap

In [ ]:
path_to_data = '/cw3e/mead/projects/cwp140/scratch/dnash/data/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [ ]:
fname_lst = glob.glob('../out/combined_df_*.csv')
print(fname_lst)
df_lst = []
df_lst2 = [] # whole dataset
for i, fname in enumerate(fname_lst):   
    df = pd.read_csv(fname)
    
    ## clean freezing level - set to nan where freezing level > 10000 m
    df.loc[(df.freezing_level > 10000), 'freezing_level'] = np.nan 
    
    ## clean ASOS_1hr - set to nan where ASOS_1hr > 200 mm per hour
    df.loc[(df.ASOS_1hr > 50), 'ASOS_1hr'] = np.nan
    df.loc[((df.ASOS_prec_accum > 600) | (df.ASOS_prec_accum < 0)), 'ASOS_prec_accum'] = np.nan 
    
    df_lst2.append(df)
    df = df.loc[df['impact_scale'] >= 0]
    
    df_lst.append(df)
    
result = pd.concat(df_lst)
result

In [ ]:
ARID = int(result.loc[result['impact_scale'] == 5].iloc[0]['trackID'])
start_date = result.loc[result['impact_scale'] == 5].iloc[0]['start_date']
end_date = result.loc[result['impact_scale'] == 5].iloc[0]['end_date']
print(ARID, start_date, end_date)

In [ ]:
ext = [-180., -120., 20, 70]
## open IVT file
ds = ar_funcs.read_GEFSv12_reforecast_data('ivt', ARID)
ds = ds.sel(time=slice(start_date, end_date), lon=slice(ext[0], ext[1]), lat=slice(ext[3], ext[2]))
ds = ds.mean('time')
ds

In [ ]:
# Set up projection
# mapcrs = ccrs.Mercator()
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
lats = ds.lat.values
lons = ds.lon.values
dx = np.arange(lons.min().round(),lons.max().round()+20,20)
dy = np.arange(lats.min().round(),lats.max().round()+10,10)



# list of letters to append to titles
letter_lst = list(map(chr, range(97, 123)))

In [ ]:
dx

In [ ]:
# Create figure
fig = plt.figure(figsize=(5, 5))
fig.dpi = 300
fname = path_to_figs + 'IVT_{0}'.format(ARID)
fmt = 'png'

nrows = 1
ncols = 1


# Set up Axes Grid
axes_class = (GeoAxes,dict(projection=mapcrs))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.2,
                cbar_location='bottom', cbar_mode='single',
                cbar_pad=0.05, cbar_size='3%',label_mode='keep')

for k, ax in enumerate(axgr):

    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=True, right_lats=False, bottom_lons=True)

    # Contour Filled
    lats = ds.lat.values
    lons = ds.lon.values
    ivt = ds.ivt.values
    cflevs = np.arange(100, 750, 50) # levels for IVT
    cmap = cmo.deep # cmap for IVT
    cf = ax.contourf(lons, lats, ivt, transform=datacrs,
                     levels=cflevs, cmap=cmap, alpha=0.9, extend='max')

    # Wind barbs / vectors 
    uvec = ds.ivtu.values
    vvec = ds.ivtv.values
    uvec_mask = ds.ivtu.where((ds.ivt >=250.)).values # mask values where IVT magnitude is less than 250 kg m-1 s-1
    vvec_mask = ds.ivtv.where((ds.ivt >=250.)).values # mask values where IVT magnitude is less than 250 kg m-1 s-1

    Q = ax.quiver(lons, lats, uvec_mask, vvec_mask, transform=datacrs, 
              color='k', regrid_shape=20,
              angles='xy', scale_units='xy', scale=125, units='xy')

    # quiver key
    qk = ax.quiverkey(Q, 0.8, 0.025, 250, '250 kg m$^{-1}$ s$^{-1}$', labelpos='E',
                      coordinates='axes', fontproperties={'size': 6.0})

# Colorbar (single)
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=False)
cb.set_label('IVT (kg m$^{-1}$ s$^{-1}$)', fontsize=11)
cb.ax.tick_params(labelsize=12)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

# Show
plt.show()

In [ ]:
## open freezing file
ds = ar_funcs.read_GEFSv12_reforecast_data('freezing_level', ARID)
ds = ds.sel(time=slice(start_date, end_date), lon=slice(ext[0], ext[1]), lat=slice(ext[3], ext[2]))

In [ ]:
# Create figure
fig = plt.figure(figsize=(5, 5))
fig.dpi = 300
fname = path_to_figs + 'IVT_{0}'.format(ARID)
fmt = 'png'

nrows = 1
ncols = 1


# Set up Axes Grid
axes_class = (GeoAxes,dict(projection=mapcrs))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.2,
                cbar_location='bottom', cbar_mode='single',
                cbar_pad=0.05, cbar_size='3%',label_mode='keep')

for k, ax in enumerate(axgr):

    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=True, right_lats=False, bottom_lons=True)

    # Contour Filled
    lats = ds.lat.values
    lons = ds.lon.values
    data = ds.freezing_level.isel(time=40).values
    cflevs = np.arange(0, 6000, 500)
    cmap = cmo.dense 
    cf = ax.contourf(lons, lats, data, transform=datacrs,
                     levels=cflevs, cmap=cmap, alpha=0.9, extend='max')

# Colorbar (single)
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=False)
cb.set_label('Freezing Level (m)', fontsize=11)
cb.ax.tick_params(labelsize=12)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

# Show
plt.show()